In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：2


['https://www.auveco.com/products/assortments/cabinetandcompartmentbinassortments',
 'https://www.auveco.com/products/bodyhardware/boltsbodyhardware/bodybolts']

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param.pop('page', '')
dict_param['show'] = 48
dict_param['display'] = 'list'

url = url.split('?')[0] + '?'
for key, value in dict_param.items():
    url += f'{key}={value}&'
    
list_url, i = [], 1
while True:
    b = 0
    while True:
        b += 1
        print(b, i)
        try:
            resp = requests.get(f'{url}page={i}',
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))
        
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                list_href = html.xpath('//div[@class="facets-facet-browse-items"]/div[@class="facets-items-collection-view-row"]/div[@class="facets-items-collection-view-cell-span12"]/div[@itemprop="itemListElement"]/div[@class="facets-item-cell-list-right"]/meta[@itemprop="url"]/@content')
                if list_href:
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    list_url += [f'''https://www.auveco.com{href.strip()}''' for href in list_href]

    # = = = = = = = = = = = = = = =
    
    i += 1

    # = = = = = = = = = = = = = = =

    if not html.xpath('//li[@class="global-views-pagination-next"]'):
        break

resp

1 1


<Response [200]>

In [6]:
output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                       'Url': list_url})

output = output.sort_values(by=['No'],
                            ascending=[True],
                            ignore_index=True)

output.to_excel('./test_url.xlsx', index=False)

output

,No,Url
0,1,https://www.auveco.com/metric-fastener-assortm...
1,2,https://www.auveco.com/metric-fastener-assortm...
2,3,https://www.auveco.com/metric-fastener-asst-re...
3,4,https://www.auveco.com/metric-fast-asst-in-40-...
4,5,https://www.auveco.com/uss-fastener-asst-wo-bi...
5,6,https://www.auveco.com/gr-8-fastener-asst-in-4...
6,7,https://www.auveco.com/gr-5-uss-fast-asst-w-o-...
7,8,https://www.auveco.com/gr-5-fast-asst-in-40-co...
8,9,https://www.auveco.com/import-retainer-assortm...
9,10,https://www.auveco.com/import-retainer-assortm...


In [7]:
crawler_status = 'ok'

crawler_status

'ok'

In [8]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [9]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel('./test_url.xlsx', index=False)

output_error

""
